In [1]:
import os
import numpy as np
from skimage import io
from PIL import Image, ImageEnhance, ImageFilter
import random
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # Sử dụng TensorFlow Keras


2024-11-01 23:19:26.577105: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-01 23:19:26.585979: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-01 23:19:26.595888: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-01 23:19:26.598883: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-01 23:19:26.606639: I tensorflow/core/platform/cpu_feature_guar

In [2]:
# Thiết lập ImageDataGenerator cho các biến đổi cơ bản
datagen = ImageDataGenerator(
    rotation_range=45,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

In [3]:
# Các hàm hỗ trợ cho các kỹ thuật tăng cường khác
def random_crop(image, size=(224, 224)):
    width, height = image.size
    new_width, new_height = size
    left = random.randint(0, width - new_width)
    top = random.randint(0, height - new_height)
    return image.crop((left, top, left + new_width, top + new_height))

In [4]:
def adjust_brightness(image):
    enhancer = ImageEnhance.Brightness(image)
    return enhancer.enhance(random.uniform(0.5, 1.5))

In [5]:

def adjust_contrast(image):
    enhancer = ImageEnhance.Contrast(image)
    return enhancer.enhance(random.uniform(0.5, 1.5))

In [6]:
def adjust_saturation(image):
    enhancer = ImageEnhance.Color(image)
    return enhancer.enhance(random.uniform(0.5, 1.5))

In [7]:
def adjust_hue(image):
    np_image = np.array(image.convert('HSV'))
    np_image[:, :, 0] = np_image[:, :, 0] + random.randint(-10, 10)
    return Image.fromarray(np_image, 'HSV').convert('RGB')

In [8]:
def add_noise(image):
    np_image = np.array(image)
    noise = np.random.normal(0, 25, np_image.shape)
    np_image = np.clip(np_image + noise, 0, 255).astype(np.uint8)
    return Image.fromarray(np_image)

In [9]:
def blur_image(image):
    return image.filter(ImageFilter.GaussianBlur(radius=random.uniform(0.5, 2.0)))


In [10]:
def adjust_sharpness(image):
    enhancer = ImageEnhance.Sharpness(image)
    return enhancer.enhance(random.uniform(0.5, 2.0))

In [11]:
def grayscale(image):
    return image.convert('L').convert('RGB')

In [12]:
def cutout(image, num_patches=1, patch_size=32):
    np_image = np.array(image)
    for _ in range(num_patches):
        x = random.randint(0, np_image.shape[1] - patch_size)
        y = random.randint(0, np_image.shape[0] - patch_size)
        np_image[y:y+patch_size, x:x+patch_size] = 0
    return Image.fromarray(np_image)

In [13]:

def elastic_deformation(image):
    return image  # Placeholder cho biến dạng đàn hồi

In [14]:
# Hàm để tăng cường ảnh
def augment_image(img):
    augmented_images = []
    
    # Các biến đổi cơ bản
    augmented_images.append(datagen.random_transform(img))  # Tăng cường bằng ImageDataGenerator
    
    # Các kỹ thuật khác
    pil_img = Image.fromarray(img)
    augmented_images.append(np.array(random_crop(pil_img)))  # Cắt ngẫu nhiên
    augmented_images.append(np.array(adjust_brightness(pil_img)))  # Điều chỉnh độ sáng
    augmented_images.append(np.array(adjust_contrast(pil_img)))  # Điều chỉnh độ tương phản
    augmented_images.append(np.array(adjust_saturation(pil_img)))  # Điều chỉnh độ bão hòa
    augmented_images.append(np.array(adjust_hue(pil_img)))  # Điều chỉnh sắc thái
    augmented_images.append(np.array(add_noise(pil_img)))  # Thêm nhiễu
    augmented_images.append(np.array(blur_image(pil_img)))  # Làm mờ ảnh
    augmented_images.append(np.array(adjust_sharpness(pil_img)))  # Điều chỉnh độ sắc nét
    augmented_images.append(np.array(grayscale(pil_img)))  # Chuyển đổi sang đen trắng
    augmented_images.append(np.array(cutout(pil_img)))  # Cắt các vùng ngẫu nhiên
    augmented_images.append(np.array(elastic_deformation(pil_img)))  # Biến dạng đàn hồi
    
    return augmented_images

In [15]:
# Hàm thực hiện tăng cường và lưu ảnh đầu ra
def tangCuongAnh(thuMucVao, thuMucRa):
    if not os.path.exists(thuMucRa):
        os.makedirs(thuMucRa)
    
    image_files = [f for f in os.listdir(thuMucVao) if f.endswith('.jpg') or f.endswith('.png')]
    
    for image_file in image_files:
        img_path = os.path.join(thuMucVao, image_file)
        img = io.imread(img_path)
        img = np.array(Image.fromarray(img).resize((224, 224)))
        
        # Tạo ảnh tăng cường
        augmented_images = augment_image(img)  # Gọi hàm tăng cường
        for i, aug_img in enumerate(augmented_images):  # Duyệt qua từng ảnh đã tăng cường
            aug_img_pil = Image.fromarray(aug_img)
            aug_img_pil.save(os.path.join(thuMucRa, f"{image_file.split('.')[0]}_aug_{i}.png"))  # Lưu ảnh tăng cường

In [16]:
# Thư mục đầu vào và đầu ra
ThuMucDauVao = 'dataset'
ThuMucDauRa = 'aug_dataset'
TepCon = ['High squamous intra-epithelial lesion', 'Low squamous intra-epithelial lesion', 'Negative for Intraepithelial malignancy', 'Squamous cell carcinoma']  # Các thư mục con trong thư mục gốc

In [17]:
# Thực hiện tăng cường cho từng thư mục con
for sub_dir in TepCon:
    input_path = os.path.join(ThuMucDauVao, sub_dir)
    output_path = os.path.join(ThuMucDauRa, f"{sub_dir}_aug")
    tangCuongAnh(input_path, output_path)  # Gọi hàm tăng cường

print("Hoàn thành tăng cường dữ liệu cho các giai đoạn bệnh.")

Hoàn thành tăng cường dữ liệu cho các giai đoạn bệnh.
